In [1]:
import re
import pandas as pd
from IPython.core.display import HTML

In [2]:
f = open('data/train.txt')
train_data = f.readlines()

f = open('data/test.txt')
test_data = f.readlines()

## 📌 Raw Data 전처리
> 대회에서 제공한 train/text 데이터를 학습할 데이터를 생성하기 용이한 형태로 변경하는 단계입니다.
1. ```Input``` : raw train, text 데이터
2. ```Output``` : ner tag가 제거된 text, text에 포함된 entity/tag리스트, entity 개수 

In [3]:
def preprocess_text(line):
    entities, tags = [], []
    l = re.findall(r'<[%-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·\s0-9ㄱ-ㅣ가-힣A-Za-z]+:[A-Za-z]+>', line)
    
    for label in l:
        entity, tag = label.replace('<', '').replace('>','').split(':')
        entities.append(entity)
        tags.append(tag)
        line = line.replace('\n', '').replace(label, entity)
    
    return line, entities, tags

In [4]:
def prepare_df(data):
    preprocessed_text, entities, tags, counts = [], [], [], []
    
    for line in data:
        line, entity, tag = preprocess_text(line)
        preprocessed_text.append(line)
        entities.append(entity)
        tags.append(tag)
        counts.append(len(entity))
    
    df = pd.DataFrame({"text": data, 'preprocessed_text': preprocessed_text, 'entities': entities, 'tags': tags, 'cnt': counts})
    return df

In [5]:
train_df = prepare_df(train_data)
test_df = prepare_df(test_data)

In [6]:
train_df.head()

,text,preprocessed_text,entities,tags,cnt
0,아름다운 <첫걸음:QT> . 과정없이 자라는 나무는 없어요~\n,아름다운 첫걸음 . 과정없이 자라는 나무는 없어요~,[첫걸음],[QT],1
1,TV상영종료후 <1년뒤:DT>에 극장판이 나올정도의 초 대작.TV편의 설정오류 수정...,TV상영종료후 1년뒤에 극장판이 나올정도의 초 대작.TV편의 설정오류 수정과 새로운...,[1년뒤],[DT],1
2,"지루할 틈없이 웃음포인트들이 있고, <4년전:DT> 영화지만 지금:봐도 재미있네요\n","지루할 틈없이 웃음포인트들이 있고, 4년전 영화지만 지금:봐도 재미있네요",[4년전],[DT],1
3,<고3:QT>때 개봉날 극장서 봤는데 당시 엄청 감동이였음.같이 본 여자애들 울고 ...,고3때 개봉날 극장서 봤는데 당시 엄청 감동이였음.같이 본 여자애들 울고 그랬어요.,[고3],[QT],1
4,눈물 펑펑 역시 <병만:PS>삼촌은 기대를 져버리지 않았슴다!\n,눈물 펑펑 역시 병만삼촌은 기대를 져버리지 않았슴다!,[병만],[PS],1


In [7]:
test_df.head()

,text,preprocessed_text,entities,tags,cnt
0,초반에 약간 뭐하는거지 할수는있는데뒤로갈수록 몰입도장난아님 주제도좋고 진짜편집좋다 ...,초반에 약간 뭐하는거지 할수는있는데뒤로갈수록 몰입도장난아님 주제도좋고 진짜편집좋다 ...,[앤드류거필드],[PS],1
1,국내 업체들이 마진이 적다는 이유로 경차 개발을 꺼린 탓에 경차가 자동차 시장 전체...,국내 업체들이 마진이 적다는 이유로 경차 개발을 꺼린 탓에 경차가 자동차 시장 전체...,[10%],[QT],1
2,역시 <한지원:PS> 감독님.!코피루왁에서 눈물흘리면서 힐링 잘했습니다\n,역시 한지원 감독님.!코피루왁에서 눈물흘리면서 힐링 잘했습니다,[한지원],[PS],1
3,<슬라이:PS>가 너무 평법한 액션 스릴러로 풀어버린 작품이다.\n,슬라이가 너무 평법한 액션 스릴러로 풀어버린 작품이다.,[슬라이],[PS],1
4,<일본:LC> 애니메이션 진격의 거인을 연상시키는 진격의 농부가 등장해 각종 커뮤니...,일본 애니메이션 진격의 거인을 연상시키는 진격의 농부가 등장해 각종 커뮤니티 사이트...,[일본],[LC],1


## 💡 Pre-requisites
> 3가지 task 데이터 생성에 모두 필요한 딕셔너리/함수를 사전 정의합니다.

- 3가지 task의 학습 데이터는 raw 데이터에 주어진 영어 엔티티 라벨이 아닌 한글 엔티티 라벨을 사용합니다. **영어 엔티티 라벨을 한글 엔티티 라벨로 변환**하는데 사용하는데 필요한 딕셔너리입니다.

In [8]:
eng_to_kor = {'PS': '사람', 'LC': '위치', 'OG': '기관', 'DT': '날짜', 'TI': '시간', 'QT': '수량'}

- NER task, Entity Extraction task의 Output은 '엔티티1은 태그1이고, ..., 엔티티N은 태그N이다.' 형태의 문장입니다. 단어의 종성에 따라 뒤따라오는 조사의 형태가 다르므로 **엔티티/태그 단어의 종성에 따라 적합한 조사를 붙여서 반환**하기 위해 필요한 함수입니다.

In [9]:
NO_JONGSUNG = 'ᴕ'

CHOSUNGS = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
JOONGSUNGS = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
JONGSUNGS = [NO_JONGSUNG,  'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

N_CHOSUNGS = 19
N_JOONGSUNGS = 21
N_JONGSUNGS = 28

FIRST_HANGUL = 0xAC00 #'가'
LAST_HANGUL = 0xD7A3 #'힣'

def get_josa(s, s_type):        
    result = []
    for c in s:
        if ord(c) < FIRST_HANGUL or ord(c) > LAST_HANGUL: # if a character is a hangul
            result.append(c)
        else:            
            code = ord(c) - FIRST_HANGUL
            jongsung_index = code % N_JONGSUNGS
            code //= N_JONGSUNGS
            joongsung_index = code % N_JOONGSUNGS
            code //= N_JOONGSUNGS
            chosung_index = code

            result.append(CHOSUNGS[chosung_index])
            result.append(JOONGSUNGS[joongsung_index])
            result.append(JONGSUNGS[jongsung_index])
    jaso_str = ''.join(result)
    if s_type == 'ent':
        josa = '는' if jaso_str[-1] == 'ᴕ' else '은'  # 종성 없으면 '는' 있으면 '은'
    if s_type == 'tag':
        josa = '' if jaso_str[-1] == 'ᴕ' else '이'  # 종성 없으면 '' 있으면 '이' 
    return josa

## [1] Named Entity Recognition 데이터 생성
1. ```Input``` : ①단계를 통해 전처리가 완료된 데이터
2. ```Output``` : NER Task에 맞게 가공된 학습 데이터
    - **NER Input** : Sentence: 문장 Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량
    - **NER Output** : 엔티티1은 태그1이고, 엔티티2는 태그2이고, ..., 엔티티N은 태그N이다.

In [10]:
def prepare_ner_data(df):
    inputs, outputs = [], []
    entities, tags = df['entities'], df['tags']
    instruction = ' Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요.'+\
                    ' 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량'

    for i, text in enumerate(df['preprocessed_text']):
        input_sentence = 'Sentence: ' + text + instruction
        output_sentence = ''
        kor_tags = [eng_to_kor[tag] for tag in tags[i]]
        for idx, entity in enumerate(entities[i]):
            output_sentence += (entity+get_josa(entity, 'ent')+' ')
            if idx == len(entities[i])-1:
                output_sentence += (kor_tags[idx]+get_josa(kor_tags[idx], 'tag')+'다.')
            else:
                output_sentence += (kor_tags[idx]+get_josa(kor_tags[idx], 'tag')+'고, ')
        inputs.append(input_sentence)
        outputs.append(output_sentence)
        
    df = pd.DataFrame({"inputs": inputs, 'outputs': outputs})
    return df

In [11]:
ner_data = prepare_ner_data(train_df)

In [12]:
display(HTML(ner_data[:5].to_html()))

,inputs,outputs
0,"Sentence: 아름다운 첫걸음 . 과정없이 자라는 나무는 없어요~ Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",첫걸음은 수량이다.
1,"Sentence: TV상영종료후 1년뒤에 극장판이 나올정도의 초 대작.TV편의 설정오류 수정과 새로운 씬 추가가 주된 볼거리. 마마마는 어른들을 위한 애니이다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",1년뒤는 날짜다.
2,"Sentence: 지루할 틈없이 웃음포인트들이 있고, 4년전 영화지만 지금:봐도 재미있네요 Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",4년전은 날짜다.
3,"Sentence: 고3때 개봉날 극장서 봤는데 당시 엄청 감동이였음.같이 본 여자애들 울고 그랬어요. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",고3은 수량이다.
4,"Sentence: 눈물 펑펑 역시 병만삼촌은 기대를 져버리지 않았슴다! Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",병만은 사람이다.


In [13]:
display(HTML(ner_data[-5:].to_html()))

,inputs,outputs
20797,"Sentence: 현재 페이스북 메신저 영상통화가 가능한 나라는 벨기에, 캐나다, 크로아티아, 덴마크, 프랑스, 그리스, 아일랜드, 라오스, 리투아니아, 멕시코, 나이지리아, 노르웨이, 오만, 폴란드, 포르투갈, 영국, 미국, 우루과이 18개국이다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","벨기에는 위치고, 캐나다는 위치고, 크로아티아는 위치고, 덴마크는 위치고, 프랑스는 위치고, 그리스는 위치고, 아일랜드는 위치고, 라오스는 위치고, 리투아니아는 위치고, 멕시코는 위치고, 나이지리아는 위치고, 노르웨이는 위치고, 오만은 위치고, 폴란드는 위치고, 포르투갈은 위치고, 영국은 위치고, 미국은 위치고, 우루과이는 위치고, 18개국은 수량이다."
20798,"Sentence: 피격 항공편 탑승자의 국적은 네덜란드가 189명으로 가장 많고, 이어 말레이시아 29명, 호주 27명, 인도네시아 12명, 영국 9명, 독일과 벨기에가 각각 4명, 필리핀과 베트남이 각각 3명, 캐나다와 뉴질랜드, 미국이 각각 1명으로 파악됐다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","네덜란드는 위치고, 189명은 수량이고, 말레이시아는 위치고, 29명은 수량이고, 호주는 위치고, 27명은 수량이고, 인도네시아는 위치고, 12명은 수량이고, 영국은 위치고, 9명은 수량이고, 독일은 위치고, 벨기에는 위치고, 4명은 수량이고, 필리핀은 위치고, 베트남은 위치고, 3명은 수량이고, 캐나다는 위치고, 뉴질랜드는 위치고, 미국은 위치고, 1명은 수량이다."
20799,"Sentence: 이날 오전 6시 현재 서울 영하 2.8도, 파주 영하 8.3도, 인천 영하 0.9도, 수원 영하 4.1도, 대전 영하 4.6도, 전주 영하 1.6도, 광주 영하 3.4도, 대구 영하 1.4도, 부산 영하 0.3도 등을 기록했다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","이날은 날짜고, 오전 6시는 시간이고, 서울은 위치고, 영하 2.8도는 수량이고, 파주는 위치고, 영하 8.3도는 수량이고, 인천은 위치고, 영하 0.9도는 수량이고, 수원은 위치고, 영하 4.1도는 수량이고, 대전은 위치고, 영하 4.6도는 수량이고, 전주는 위치고, 영하 1.6도는 수량이고, 광주는 위치고, 영하 3.4도는 수량이고, 대구는 위치고, 영하 1.4도는 수량이고, 부산은 위치고, 영하 0.3은 수량이다."
20800,"Sentence: 기상청에 따르면 12일 아침 서울/수원 영하 8도, 인천 영하 7도, 파주/이천 영하 12도, 동두천 영하 11도, 철원 영하 14도, 춘천/원주 영하 10도, 대관령 영하 15도 등으로 예상된다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","기상청은 기관이고, 12일은 날짜고, 아침은 시간이고, 서울은 위치고, 수원은 위치고, 8도는 수량이고, 인천은 위치고, 7도는 수량이고, 파주는 위치고, 이천은 위치고, 12도는 수량이고, 동두천은 위치고, 11도는 수량이고, 철원은 위치고, 14도는 수량이고, 춘천은 위치고, 원주는 위치고, 10도는 수량이고, 대관령은 위치고, 15도는 수량이다."
20801,"Sentence: 낮 최고기온은 서울 9도, 인천 7도, 춘천 9도, 강릉 10도, 청주 12도, 대전 12도, 전주 11도, 광주 11도, 대구 12도, 부산 12도, 제주 16도 등이다. Instruction: Input Sentence에서 찾을 수 있는 모든 Entity 및 그들의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","낮은 시간이고, 서울은 위치고, 9도는 수량이고, 인천은 위치고, 7도는 수량이고, 춘천은 위치고, 9도는 수량이고, 강릉은 위치고, 10도는 수량이고, 청주는 위치고, 12도는 수량이고, 대전은 위치고, 12도는 수량이고, 전주는 위치고, 11도는 수량이고, 광주는 위치고, 11도는 수량이고, 대구는 위치고, 12도는 수량이고, 부산은 위치고, 12도는 수량이고, 제주는 위치고, 16도는 수량이다."


## [2] Entity Typing 데이터 생성
> ***Entity Typing***: 문장과 문장에 존재하는 엔티티가 주어진 상태에서 엔티티의 유형을 예측하는 task

1. ```Input``` : ①단계를 통해 전처리가 완료된 데이터
2. ```Output``` : Entity Typing Task에 맞게 가공된 학습 데이터
    - **ET Input** : Sentence: 문장 Instruction: Input Sentence에서 <엔티티1, 엔티티2, ..., 엔티티N>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량
    - **ET Output** : 엔티티1은 태그1이고, 엔티티2는 태그2이고, ..., 엔티티N은 태그N이다.

In [14]:
def prepare_et_data(df):
    inputs, outputs = [], []
    entities, tags = df['entities'], df['tags']

    for i, text in enumerate(df['preprocessed_text']):
        input_entities = ', '.join(entities[i])
        input_sentence = 'Sentence: ' + text + ' Instruction: Input Sentence에서 <' + input_entities + \
                            '>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량'
        output_sentence = ''
        kor_tags = [eng_to_kor[tag] for tag in tags[i]]
        for idx, entity in enumerate(entities[i]):
            output_sentence += (entity+get_josa(entity, 'ent')+' ')
            if idx == len(entities[i])-1:
                output_sentence += (kor_tags[idx]+get_josa(kor_tags[idx], 'tag')+'다.')
            else:
                output_sentence += (kor_tags[idx]+get_josa(kor_tags[idx], 'tag')+'고, ')
        inputs.append(input_sentence)
        outputs.append(output_sentence)
        
    df = pd.DataFrame({"inputs": inputs, 'outputs': outputs})
    return df

In [15]:
et_data = prepare_et_data(train_df)

In [16]:
display(HTML(et_data[:5].to_html()))

,inputs,outputs
0,"Sentence: 아름다운 첫걸음 . 과정없이 자라는 나무는 없어요~ Instruction: Input Sentence에서 <첫걸음>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",첫걸음은 수량이다.
1,"Sentence: TV상영종료후 1년뒤에 극장판이 나올정도의 초 대작.TV편의 설정오류 수정과 새로운 씬 추가가 주된 볼거리. 마마마는 어른들을 위한 애니이다. Instruction: Input Sentence에서 <1년뒤>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",1년뒤는 날짜다.
2,"Sentence: 지루할 틈없이 웃음포인트들이 있고, 4년전 영화지만 지금:봐도 재미있네요 Instruction: Input Sentence에서 <4년전>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",4년전은 날짜다.
3,"Sentence: 고3때 개봉날 극장서 봤는데 당시 엄청 감동이였음.같이 본 여자애들 울고 그랬어요. Instruction: Input Sentence에서 <고3>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",고3은 수량이다.
4,"Sentence: 눈물 펑펑 역시 병만삼촌은 기대를 져버리지 않았슴다! Instruction: Input Sentence에서 <병만>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량",병만은 사람이다.


In [17]:
display(HTML(et_data[-5:].to_html()))

,inputs,outputs
20797,"Sentence: 현재 페이스북 메신저 영상통화가 가능한 나라는 벨기에, 캐나다, 크로아티아, 덴마크, 프랑스, 그리스, 아일랜드, 라오스, 리투아니아, 멕시코, 나이지리아, 노르웨이, 오만, 폴란드, 포르투갈, 영국, 미국, 우루과이 18개국이다. Instruction: Input Sentence에서 <벨기에, 캐나다, 크로아티아, 덴마크, 프랑스, 그리스, 아일랜드, 라오스, 리투아니아, 멕시코, 나이지리아, 노르웨이, 오만, 폴란드, 포르투갈, 영국, 미국, 우루과이, 18개국>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","벨기에는 위치고, 캐나다는 위치고, 크로아티아는 위치고, 덴마크는 위치고, 프랑스는 위치고, 그리스는 위치고, 아일랜드는 위치고, 라오스는 위치고, 리투아니아는 위치고, 멕시코는 위치고, 나이지리아는 위치고, 노르웨이는 위치고, 오만은 위치고, 폴란드는 위치고, 포르투갈은 위치고, 영국은 위치고, 미국은 위치고, 우루과이는 위치고, 18개국은 수량이다."
20798,"Sentence: 피격 항공편 탑승자의 국적은 네덜란드가 189명으로 가장 많고, 이어 말레이시아 29명, 호주 27명, 인도네시아 12명, 영국 9명, 독일과 벨기에가 각각 4명, 필리핀과 베트남이 각각 3명, 캐나다와 뉴질랜드, 미국이 각각 1명으로 파악됐다. Instruction: Input Sentence에서 <네덜란드, 189명, 말레이시아, 29명, 호주, 27명, 인도네시아, 12명, 영국, 9명, 독일, 벨기에, 4명, 필리핀, 베트남, 3명, 캐나다, 뉴질랜드, 미국, 1명>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","네덜란드는 위치고, 189명은 수량이고, 말레이시아는 위치고, 29명은 수량이고, 호주는 위치고, 27명은 수량이고, 인도네시아는 위치고, 12명은 수량이고, 영국은 위치고, 9명은 수량이고, 독일은 위치고, 벨기에는 위치고, 4명은 수량이고, 필리핀은 위치고, 베트남은 위치고, 3명은 수량이고, 캐나다는 위치고, 뉴질랜드는 위치고, 미국은 위치고, 1명은 수량이다."
20799,"Sentence: 이날 오전 6시 현재 서울 영하 2.8도, 파주 영하 8.3도, 인천 영하 0.9도, 수원 영하 4.1도, 대전 영하 4.6도, 전주 영하 1.6도, 광주 영하 3.4도, 대구 영하 1.4도, 부산 영하 0.3도 등을 기록했다. Instruction: Input Sentence에서 <이날, 오전 6시, 서울, 영하 2.8도, 파주, 영하 8.3도, 인천, 영하 0.9도, 수원, 영하 4.1도, 대전, 영하 4.6도, 전주, 영하 1.6도, 광주, 영하 3.4도, 대구, 영하 1.4도, 부산, 영하 0.3>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","이날은 날짜고, 오전 6시는 시간이고, 서울은 위치고, 영하 2.8도는 수량이고, 파주는 위치고, 영하 8.3도는 수량이고, 인천은 위치고, 영하 0.9도는 수량이고, 수원은 위치고, 영하 4.1도는 수량이고, 대전은 위치고, 영하 4.6도는 수량이고, 전주는 위치고, 영하 1.6도는 수량이고, 광주는 위치고, 영하 3.4도는 수량이고, 대구는 위치고, 영하 1.4도는 수량이고, 부산은 위치고, 영하 0.3은 수량이다."
20800,"Sentence: 기상청에 따르면 12일 아침 서울/수원 영하 8도, 인천 영하 7도, 파주/이천 영하 12도, 동두천 영하 11도, 철원 영하 14도, 춘천/원주 영하 10도, 대관령 영하 15도 등으로 예상된다. Instruction: Input Sentence에서 <기상청, 12일, 아침, 서울, 수원, 8도, 인천, 7도, 파주, 이천, 12도, 동두천, 11도, 철원, 14도, 춘천, 원주, 10도, 대관령, 15도>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","기상청은 기관이고, 12일은 날짜고, 아침은 시간이고, 서울은 위치고, 수원은 위치고, 8도는 수량이고, 인천은 위치고, 7도는 수량이고, 파주는 위치고, 이천은 위치고, 12도는 수량이고, 동두천은 위치고, 11도는 수량이고, 철원은 위치고, 14도는 수량이고, 춘천은 위치고, 원주는 위치고, 10도는 수량이고, 대관령은 위치고, 15도는 수량이다."
20801,"Sentence: 낮 최고기온은 서울 9도, 인천 7도, 춘천 9도, 강릉 10도, 청주 12도, 대전 12도, 전주 11도, 광주 11도, 대구 12도, 부산 12도, 제주 16도 등이다. Instruction: Input Sentence에서 <낮, 서울, 9도, 인천, 7도, 춘천, 9도, 강릉, 10도, 청주, 12도, 대전, 12도, 전주, 11도, 광주, 11도, 대구, 12도, 부산, 12도, 제주, 16도>의 Entity type을 출력하세요. 가능한 Entity type은 다음과 같습니다: 사람, 위치, 기관, 날짜, 시간, 수량","낮은 시간이고, 서울은 위치고, 9도는 수량이고, 인천은 위치고, 7도는 수량이고, 춘천은 위치고, 9도는 수량이고, 강릉은 위치고, 10도는 수량이고, 청주는 위치고, 12도는 수량이고, 대전은 위치고, 12도는 수량이고, 전주는 위치고, 11도는 수량이고, 광주는 위치고, 11도는 수량이고, 대구는 위치고, 12도는 수량이고, 부산은 위치고, 12도는 수량이고, 제주는 위치고, 16도는 수량이다."


## [3] Entity Extraction 데이터 생성
> ***Entity Extraction***: 문장이 주어졌을 때 문장에 존재하는 엔티티를 찾아내는 task

1. ```Input``` : ①단계를 통해 전처리가 완료된 데이터
2. ```Output``` : Entity Extraction Task에 맞게 가공된 학습 데이터
    - **EE Input** : Sentence: <문장> Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요. 
    - **EE Output** : 엔티티1, 엔티티2, ..., 엔티티N.

In [18]:
def prepare_ee_data(df):
    inputs, outputs = [], []
    entities, tags = df['entities'], df['tags']
    instruction = ' Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.'

    for i, text in enumerate(df['preprocessed_text']):
        input_sentence = 'Sentence: ' + text + instruction
        output_sentence = ', '.join(entities[i]) + '.'
        inputs.append(input_sentence)
        outputs.append(output_sentence)
        
    df = pd.DataFrame({"inputs": inputs, 'outputs': outputs})
    return df

In [19]:
ee_data = prepare_ee_data(train_df)

In [20]:
display(HTML(ee_data[:5].to_html()))

,inputs,outputs
0,Sentence: 아름다운 첫걸음 . 과정없이 자라는 나무는 없어요~ Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.,첫걸음.
1,Sentence: TV상영종료후 1년뒤에 극장판이 나올정도의 초 대작.TV편의 설정오류 수정과 새로운 씬 추가가 주된 볼거리. 마마마는 어른들을 위한 애니이다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.,1년뒤.
2,"Sentence: 지루할 틈없이 웃음포인트들이 있고, 4년전 영화지만 지금:봐도 재미있네요 Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.",4년전.
3,Sentence: 고3때 개봉날 극장서 봤는데 당시 엄청 감동이였음.같이 본 여자애들 울고 그랬어요. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.,고3.
4,Sentence: 눈물 펑펑 역시 병만삼촌은 기대를 져버리지 않았슴다! Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.,병만.


In [21]:
display(HTML(ee_data[-5:].to_html()))

,inputs,outputs
20797,"Sentence: 현재 페이스북 메신저 영상통화가 가능한 나라는 벨기에, 캐나다, 크로아티아, 덴마크, 프랑스, 그리스, 아일랜드, 라오스, 리투아니아, 멕시코, 나이지리아, 노르웨이, 오만, 폴란드, 포르투갈, 영국, 미국, 우루과이 18개국이다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.","벨기에, 캐나다, 크로아티아, 덴마크, 프랑스, 그리스, 아일랜드, 라오스, 리투아니아, 멕시코, 나이지리아, 노르웨이, 오만, 폴란드, 포르투갈, 영국, 미국, 우루과이, 18개국."
20798,"Sentence: 피격 항공편 탑승자의 국적은 네덜란드가 189명으로 가장 많고, 이어 말레이시아 29명, 호주 27명, 인도네시아 12명, 영국 9명, 독일과 벨기에가 각각 4명, 필리핀과 베트남이 각각 3명, 캐나다와 뉴질랜드, 미국이 각각 1명으로 파악됐다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.","네덜란드, 189명, 말레이시아, 29명, 호주, 27명, 인도네시아, 12명, 영국, 9명, 독일, 벨기에, 4명, 필리핀, 베트남, 3명, 캐나다, 뉴질랜드, 미국, 1명."
20799,"Sentence: 이날 오전 6시 현재 서울 영하 2.8도, 파주 영하 8.3도, 인천 영하 0.9도, 수원 영하 4.1도, 대전 영하 4.6도, 전주 영하 1.6도, 광주 영하 3.4도, 대구 영하 1.4도, 부산 영하 0.3도 등을 기록했다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.","이날, 오전 6시, 서울, 영하 2.8도, 파주, 영하 8.3도, 인천, 영하 0.9도, 수원, 영하 4.1도, 대전, 영하 4.6도, 전주, 영하 1.6도, 광주, 영하 3.4도, 대구, 영하 1.4도, 부산, 영하 0.3."
20800,"Sentence: 기상청에 따르면 12일 아침 서울/수원 영하 8도, 인천 영하 7도, 파주/이천 영하 12도, 동두천 영하 11도, 철원 영하 14도, 춘천/원주 영하 10도, 대관령 영하 15도 등으로 예상된다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.","기상청, 12일, 아침, 서울, 수원, 8도, 인천, 7도, 파주, 이천, 12도, 동두천, 11도, 철원, 14도, 춘천, 원주, 10도, 대관령, 15도."
20801,"Sentence: 낮 최고기온은 서울 9도, 인천 7도, 춘천 9도, 강릉 10도, 청주 12도, 대전 12도, 전주 11도, 광주 11도, 대구 12도, 부산 12도, 제주 16도 등이다. Instruction: Input Sentence에서 Entity에 해당하는 단어를 모두 출력하세요.","낮, 서울, 9도, 인천, 7도, 춘천, 9도, 강릉, 10도, 청주, 12도, 대전, 12도, 전주, 11도, 광주, 11도, 대구, 12도, 부산, 12도, 제주, 16도."
